In [1]:
import json
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import networkx as nx
from dateutil import relativedelta
from datetime import datetime
import re
import tqdm

In [2]:

con = sqlite3.connect('../data/interim/articles_with_author_mapping.db')
cur = con.cursor()


In [3]:
cur.execute("select id, name, abbreviation, matching_certainty from authors where abbreviation is not null")
rows = cur.fetchall()
authors = pd.DataFrame(columns=["id", "name", "abbreviation", "certainty"], data=rows)
authors.set_index("id", inplace=True)

In [4]:
# convert null to nans
authors = authors.replace("null", np.nan)

In [5]:
# set certainty of a row to the average certainty over all rows with same name and abbreviation, ignore nan values
mean_certainty = authors.groupby(["name", "abbreviation"])["certainty"].transform(lambda x: np.nanmean(x))
authors["certainty"] = mean_certainty

In [6]:
authors["name_pointing_to_abbreviation_count"] = authors.groupby(["name", "abbreviation"])["name"].transform("count")
authors["abbreviation_pointing_to_name_count"] = authors.groupby(["name", "abbreviation"])["abbreviation"].transform("count")

In [7]:
# drop duplicates based on name, abbreviation, certainty. I can drop them because ..._count saved the count
authors.drop_duplicates(subset=["name", "abbreviation", "certainty"], inplace=True)

In [8]:
# set the share that the name has of all names that point to that abbreviation
authors_with_name_pointing_to_abbreviation_sum =  authors.groupby(["abbreviation"]).agg(names_pointing_to_abbreviation_sum=("name_pointing_to_abbreviation_count", "sum")).reset_index()
authors = pd.merge(authors, authors_with_name_pointing_to_abbreviation_sum, on="abbreviation")
authors["name_pointing_to_abbreviation_share"] = authors["name_pointing_to_abbreviation_count"] / authors["names_pointing_to_abbreviation_sum"]

In [9]:
# set the share that the abbreviation has of all abbreviations that point to that name
authors_with_abbreviation_pointing_to_name_sum = authors.groupby(["name"]).agg(abbreviations_pointing_to_name_sum=("abbreviation_pointing_to_name_count", "sum")).reset_index()
authors = pd.merge(authors, authors_with_abbreviation_pointing_to_name_sum, on="name")
authors["abbreviation_pointing_to_name_share"] = authors["abbreviation_pointing_to_name_count"] / authors["abbreviations_pointing_to_name_sum"]

In [10]:
# calculate a final score for the weighted edge between name and abbreviation
authors["score"] = authors["certainty"] + authors["name_pointing_to_abbreviation_share"] + authors["abbreviation_pointing_to_name_share"]

In [11]:
# reduce the problem space to a bipartite graph, assigning all (name, abbreviation) pairs that are not connected to the graph as final/solved
# add pairs to final mapping where abbreviation_pointing_to_name_count == 1 or name_pointing_to_abbreviation_count == 1 and remove those from the authors 
#one_to_one_mappings = authors[(authors["abbreviations_pointing_to_name_sum"] == 1) & (authors["names_pointing_to_abbreviation_sum"] == 1)]
#final_mapping = pd.concat([final_mapping, one_to_one_mappings[["name", "abbreviation"]]], ignore_index=True)

#authors = authors[(authors["abbreviations_pointing_to_name_sum"] != 1) | (authors["names_pointing_to_abbreviation_sum"] != 1)]

In [12]:
# remove self referencing nodes
authors = authors[authors["name"].str.lower() != authors["abbreviation"].str.lower()]

In [13]:
# add name and abbreviation as nodes list and remove duplicates
nodes = list(authors["name"].unique()) + list(authors["abbreviation"].unique())

# add unique edges based on entries in the authors table
edges = []
for index, row in authors.iterrows():
    edges.append((row["name"], row["abbreviation"]))

In [14]:
G = nx.Graph()
G.clear()
author_list = list(authors["name"].unique())
abbr_list = list(authors[authors["name"].isin(author_list)]["abbreviation"].unique())
G.add_nodes_from(author_list, bipartite=0)
G.add_nodes_from(abbr_list, bipartite=1)
dummy_nodes = [f"{name}_dummy" for name in author_list]
G.add_nodes_from(dummy_nodes, bipartite=1)

edges = []
for index, row in authors[authors["name"].isin(author_list) & authors["abbreviation"].isin(abbr_list)].iterrows():
    G.add_edges_from([(row["name"], row["abbreviation"])], weight=round(row["score"],2) * -1)

for name in author_list:
    G.add_edges_from([(name, f"{name}_dummy")], weight=0)


In [15]:
new_edges = nx.bipartite.minimum_weight_full_matching(G, top_nodes=list(authors["name"].unique()), weight="weight")
g_new = nx.Graph()
g_new.clear()
g_new.add_nodes_from(author_list, bipartite=0)
g_new.add_nodes_from(abbr_list, bipartite=1)
g_new.add_edges_from(new_edges.items())

In [16]:
g_new.edges

EdgeView([('Kai-Uwe Brandt', 'kub'), ('Nadja Topfstedt', 'jap'), ('Manfred Lüttich', 'Manfred Lüttich_dummy'), ('Magdalena Froehlich', 'mf'), ('Michael Frömmert', 'maf'), ('Heiko Trebs', 'ht'), ('Andreas Tappert', 'art'), ('Annett Riedel', 'ari'), ('Angelika Raulien', 'are'), ('Andrea Richter', 'ar'), ('Norbert Töpfer', 'nt'), ('Antje Henselin-Rudolph', 'ahr'), ('Nico Fliegner', 'nf'), ('Heike Liesaus', 'hl'), ('Juliane Streich', 'just'), ('Lisa Garn', 'lis'), ('Lisa Berins', 'la'), ('Melanie Steitz', 'mes'), ('Lisa Schliep', 'liep'), ('Kendra Reinhardt', 'kir'), ('Karoline Maria Keybe', 'Karoline Maria Keybe_dummy'), ('Karin Rieck', 'karin'), ('Karin Tamcke', 'Karin Tamcke_dummy'), ('Klaus Staeubert', 'kas'), ('Kay Würker', 'ka'), ('Kathrin Kabelitz', 'Kathrin Kabelitz_dummy'), ('Kai Kollenberg', 'kol'), ('Frank Pfütze', 'pfü'), ('Meike Strüber', 'mei'), ('Michael Dick', 'midi'), ('Manfred Hainich', 'Manfred Hainich_dummy'), ('Nikos Natsidis', 'nn'), ('Roland Heinrich', 'rohe'), ('Hei

In [17]:
print(f"new graph has {len(g_new.nodes)} nodes and {len(g_new.edges)} edges")

new graph has 681 nodes and 311 edges


In [18]:
# transform graph to dataframe author_mapping with columns name and abbreviation
author_mapping = pd.DataFrame(columns=["name", "abbreviation"], data=g_new.edges)
author_mapping = author_mapping.astype(str)

print(f"author_mapping has {author_mapping.shape[0]} rows")


# remove all abbreviations that contain "dummy" in their name
author_mapping = author_mapping[~author_mapping["abbreviation"].str.contains("dummy")]
# print shape
print(f"author_mapping has {author_mapping.shape[0]} rows")

author_mapping has 311 rows
author_mapping has 251 rows


### In the following there are some analyses covering the correctness of the approach

In [19]:
# test if there is an authors that was mapped to an abbreviation that exceeds the authors name writing time by more than 6 months in the time before or after
# Note: does not check for authors that exceed abbreviation time. Assumption: an abbreviation can be assigned to an author also later
count = 0
author_abbreviation_pairs_with_lifespan_difference = []
for index, row in author_mapping.iterrows():
    author = row["name"]
    abbreviation = row["abbreviation"]
    author_like = f"%{json.dumps(author)}%"
    abbr_like = f"%{json.dumps(abbreviation)}%"
    try:
        first_article_author = cur.execute(f"SELECT MIN(published_at) FROM articles where author_array like ?", (author_like,)).fetchone()[0]
        first_article_abbr = cur.execute(f"SELECT MIN(published_at) FROM articles where author_array like ?", (abbr_like,)).fetchone()[0]
        first_article_author = datetime.strptime(first_article_author, "%Y-%m-%dT%H:%M:%S+00:00")
        first_article_abbr = datetime.strptime(first_article_abbr, "%Y-%m-%dT%H:%M:%S+00:00")

        # check if first_article_abbr is six month ahead of first_article_author
        if first_article_abbr < first_article_author - relativedelta.relativedelta(months=12):
            print(f"author: {author}, first_article_author: {first_article_author}, abbreviation: {abbreviation}, first_article_abbr: {first_article_abbr}")
            count += 1
            author_abbreviation_pairs_with_lifespan_difference.append({"author": author, "abbreviation": abbreviation})

        last_article_author = cur.execute(f"SELECT MAX(published_at) FROM articles where author_array like ?", (author_like,)).fetchone()[0]
        last_article_abbr = cur.execute(f"SELECT MAX(published_at) FROM articles where author_array like ?", (abbr_like,)).fetchone()[0]
        last_article_author = datetime.strptime(last_article_author, "%Y-%m-%dT%H:%M:%S+00:00")
        last_article_abbr = datetime.strptime(last_article_abbr, "%Y-%m-%dT%H:%M:%S+00:00")

        # check if last_article_abbr is six month behind of last_article_author
        if last_article_abbr > last_article_author + relativedelta.relativedelta(months=12):
            print(f"author: {author}, last_article_author: {last_article_author}, abbreviation: {abbreviation}, last_article_abbr: {last_article_abbr}")
            count += 1
            author_abbreviation_pairs_with_lifespan_difference.append({"author": author, "abbreviation": abbreviation})
    except TypeError:
        print(f"author: {author}, abbreviation: {abbreviation}")
        continue

print(f"there are {count} authors that were mapped to an abbreviation that exceeds the authors name writing time by more than 12 months in the time before or after")

KeyboardInterrupt: 

We can conclude that we need a mapping that takes into account the authors' lifespans. Abbreviations can be used for more than one author. Furthermore, we need to penalize abbr-author mappings where the abbreviation has a much longer or shorter lifespan.

Further, we notice that there are abbreviations that do not have an associated author. E.g. "joka", does not have any good fit.

In [40]:
for row in tqdm.tqdm([row for _, row in author_mapping.iterrows()]):
    abbr_authors = []
    abbr = row["abbreviation"]
    mapped_author = row["name"]
    abbr_neighbours = list(G.neighbors(abbr))
    for author in abbr_neighbours:
        author_like = f"%{json.dumps(author)}%"
        rows = cur.execute('select ar.published_at from articles ar where ar.author_array like ?', (author_like,)).fetchall()
        type = "mapped name" if author == mapped_author else "name"
        abbr_authors.extend([{"author": author, "type": type, "published_at": row[0]} for row in rows])

    
    abbr_like = f"%{json.dumps(abbr)}%"
    rows = cur.execute('select ar.published_at from articles ar where ar.author_array like ?', (abbr_like,)).fetchall()
    abbr_authors.extend([{"author": abbr, "type": "abbreviation", "published_at": row[0]} for row in rows])

    abbr_authors = pd.DataFrame(columns=["author", "type", "published_at"], data=abbr_authors)

    # aggregate by quarter of the year
    abbr_authors['published_at'] = pd.to_datetime(abbr_authors['published_at'])
    abbr_authors['quarter'] = abbr_authors['published_at'].dt.quarter
    abbr_authors['year'] = abbr_authors['published_at'].dt.year
    abbr_authors['year_quarter'] = abbr_authors['year'].astype(str) + "/" + abbr_authors['quarter'].astype(str)
    #abbr_authors['year_semester'] = abbr_authors['year'].astype(str) + "/" + abbr_authors['quarter'].apply(lambda x: 1 if x <= 2 else 2).astype(str)
    abbr_authors = abbr_authors.groupby(['year_quarter', 'author', 'type']).count()['published_at'].reset_index()
    # rename published at to count
    abbr_authors = abbr_authors.rename(columns={"published_at": "count"})
    
    # fill year_quarter gaps
    year_quarters = abbr_authors['year_quarter'].drop_duplicates()
    # get min and max year
    min_year = int(min(year_quarters).split("/")[0])
    max_year = int(max(year_quarters).split("/")[0])
    # get for min_year min quarter and for max_year max quarter
    min_quarter = int(min([year_quarter.split("/")[1] for year_quarter in year_quarters if year_quarter.split("/")[0] == str(min_year)]))
    max_quarter = int(max([year_quarter.split("/")[1] for year_quarter in year_quarters if year_quarter.split("/")[0] == str(max_year)]))
    # create all year_quarters
    year_quarters = [str(year) + "/" + str(quarter) for year in range(min_year, max_year + 1) for quarter in range(1, 5)]
        
    # for each author: if there is no entry for a unique combination of 'year_quarter', add it with count np.nan
    index = pd.MultiIndex.from_product([year_quarters, abbr_authors['author'].unique()], names=['year_quarter', 'author'])
    
    # Reindex the DataFrame with the MultiIndex to fill in missing combinations with NaN
    abbr_authors = abbr_authors.set_index(['year_quarter', 'author']).reindex(index, fill_value=None).reset_index()
    
    # set type for each author to first type of that author
    abbr_authors['type'] = abbr_authors.groupby('author')['type'].transform('first')
    
    # sort by year_quarter and type
    abbr_authors = abbr_authors.sort_values(by=['year_quarter', 'type'])
    
    directory = "author_abbreviation_pairs"

    if {"author": mapped_author, "abbreviation": abbr} in author_abbreviation_pairs_with_lifespan_difference:
        directory = "author_abbreviation_pairs_with_lifespan_difference"

    
    plt.cla()
    plt.clf()
    sns.lineplot(data=abbr_authors, x='year_quarter', y='count', hue='author', style='type', errorbar=None, linewidth=2.5)
    # set y title to article count
    plt.ylabel("article count")
    plt.xlabel("year/quarter")
    sns.set(rc={'figure.figsize':(14,10)})
    plt.xticks(rotation=-75)
    # set ticks to every 4th
    plt.xticks(np.arange(0, len(abbr_authors['year_quarter'].unique()), 4))
    #plt.tight_layout()
    plt.title(f'Article count of abbreviation {abbr}, associated authors and mapped author {mapped_author}')
    plt.savefig(f'../reports/figures/{directory}/articles_written_by_abbreviation_{abbr}_and_mapped_author_{"_".join(mapped_author.split(" "))}_and_associated_authors.png',bbox_inches='tight',dpi=300)
    plt.close()

100%|██████████| 251/251 [03:10<00:00,  1.31it/s]


Mind the fact that seaborn lineplot connects non-neighbouring points. If more correct view is needed, choose scatterplot. See here for further infos: https://stackoverflow.com/questions/52098537/avoid-plotting-missing-values-on-a-line-plot


Takeaways:
* Author entities can have different names (compare Inge Engelhardt and Inge-Dore Engelhardt in "articles_written_by_abbreviation_ie_and_mapped_author_Inge_Engelhardt_and_associated_authors"
  * an obvious szenario is that the author got married and changed their name
* there are only few mappings where the abbreviation and the name share a similar article count distribution 
  * one good example is: articles_written_by_abbreviation_lyn_and_mapped_author_Evelyn_ter_Vehn_and_associated_authors
  * one example for a more or less good similarity is: articles_written_by_abbreviation_hgw_and_mapped_author_Hanna_Gerwig_and_associated_authors
  * one example for a match but with close to no similarity is: articles_written_by_abbreviation_krysta_and_mapped_author_Krysta_Brown_and_associated_authors 
  * a good example for the randomness in using the abbreviation or the name is: articles_written_by_abbreviation_mpu_and_mapped_author_Matthias_Puppe_and_associated_authors
  * TODO: Conclusion  

Note:
If an abbreviation or name has only written articles in one quarter and thus only one data point, it won't get displayed by seaborn


In [20]:
print(f"new graph has {len(g_new.nodes)} nodes and {len(g_new.edges)} edges")

new graph has 681 nodes and 311 edges


There are abbr nodes that were not matched with any author. That's why we have: edges * 2 < nodes.

In [23]:
# test that only abbrs are not in the matches nodes
for node in g_new.nodes:
    if g_new.degree(node) == 0 and node in author_list:
        print(f"author node {node} has no edges")
    elif g_new.degree(node) == 0 and node in abbr_list:
        print(f"abbreviation node {node} has no edges")

abbreviation node ast has no edges
abbreviation node at has no edges
abbreviation node ao has no edges
abbreviation node nie has no edges
abbreviation node heike has no edges
abbreviation node ks has no edges
abbreviation node kast has no edges
abbreviation node mape has no edges
abbreviation node mot has no edges
abbreviation node mabe has no edges
abbreviation node mato has no edges
abbreviation node beck has no edges
abbreviation node bos has no edges
abbreviation node seb has no edges
abbreviation node fb has no edges
abbreviation node brown has no edges
abbreviation node mayer has no edges
abbreviation node may has no edges
abbreviation node mas has no edges
abbreviation node ts has no edges
abbreviation node meine has no edges
abbreviation node thomas has no edges
abbreviation node th has no edges
abbreviation node ter vehn has no edges
abbreviation node rare has no edges
abbreviation node cha has no edges
abbreviation node ja has no edges
abbreviation node sf has no edges
abbrev

In [26]:
# list the abbreviations that were connected to the author that the edges with zero edges were pointing to
# so we can e.g. check if we need to enable multiple abbreviations for the same author
for node in g_new.nodes:
    if g_new.degree(node) == 0:
        # check the authors of the old graph that were pointing to this abbreviation
        old_neighbors = G.neighbors(node)
        for old_neighbor in old_neighbors:
            print(f"new neighbors of old abbr {node} node neighbors {old_neighbor} are {list(g_new.neighbors(old_neighbor))}")


new neighbors of old abbr ast node neighbors Andreas Tappert are ['art']
new neighbors of old abbr at node neighbors Andreas Tappert are ['art']
new neighbors of old abbr ao node neighbors Antje Henselin-Rudolph are ['ahr']
new neighbors of old abbr ao node neighbors Alexander Laboda are ['ala']
new neighbors of old abbr nie node neighbors Nico Fliegner are ['nf']
new neighbors of old abbr heike node neighbors Heike Liesaus are ['hl']
new neighbors of old abbr ks node neighbors Klaus Staeubert are ['kas']
new neighbors of old abbr ks node neighbors Katharina Stork are ['kasto']
new neighbors of old abbr kast node neighbors Klaus Staeubert are ['kas']
new neighbors of old abbr kast node neighbors Katharina Stork are ['kasto']
new neighbors of old abbr mape node neighbors Matthias Puppe are ['mpu']
new neighbors of old abbr mot node neighbors Matthias Roth are ['mro']
new neighbors of old abbr mabe node neighbors Mathias Orbeck are ['mo']
new neighbors of old abbr mabe node neighbors Mar

From this we do see that some authors have several abbreviations
For example: krysta brown has probably two abbreviations: "krysta" and "brown"

In [27]:
print(f"{(len(g_new.edges))} abbreviations were matched")
print(f"There are {len(g_new.nodes) - (len(g_new.edges) * 2)} abbreviations that were not matched")

311 abbreviations were matched
There are 59 abbreviations that were not matched


In [28]:
# print not matched abbreviations
print([node for node in g_new.nodes if g_new.degree(node) == 0])

['ast', 'at', 'ao', 'nie', 'heike', 'ks', 'kast', 'mape', 'mot', 'mabe', 'mato', 'beck', 'bos', 'seb', 'fb', 'brown', 'mayer', 'may', 'mas', 'ts', 'meine', 'thomas', 'th', 'ter vehn', 'rare', 'cha', 'ja', 'sf', 'döring', 'nag', 'fw', 'kha', 'fel', 'isc', 'ski', 'grätz', 'sag', 'rd', 'roger', 'wurzel', 'kreuz', 'sabine', 'kunze', 'büchel', 'sie', 's robak', 'iro', 'björn', 'rösner', 'haase', 'hagen', 'epd', 'pb', 'pat', 'sei', 'pelzl', 'julia', 'nq', 's helm']


In [29]:
# test if these abbreviations have only one edge in the old graph. If so, we can append them to that author.
# assumes that the names got assigned a more fitting abbreviation but these here do also belong to that name
unmatched_abbrs_with_only_one_edge = [node for node in g_new.nodes if g_new.degree(node) == 0 and len(list(G.neighbors(node))) == 1]
print(f"{len(unmatched_abbrs_with_only_one_edge)} abbreviations have only one edge in the old graph")

# list remaining abbreviations
remaining_abbrs = [node for node in g_new.nodes if g_new.degree(node) == 0 and len(list(G.neighbors(node))) > 1]
print(remaining_abbrs)

43 abbreviations have only one edge in the old graph
['ao', 'ks', 'kast', 'mabe', 'beck', 'mayer', 'may', 'mas', 'ts', 'meine', 'cha', 'nag', 'ski', 'pb', 'pat', 'sei']


We conclude, that most of the unmatched abbreviations were only connected to one author originally. That means that we can append them to that author because we know now that authors can have multiple abbreviations.